In [26]:
import pandas as pd
import sqlite3

In [27]:
connect = sqlite3.connect("raw-database.db")
cur = connect.cursor()

In [28]:
matching_matches = pd.read_json("Projet_centres_data/matching_matches.json")

In [29]:
matching_matches.to_sql("matching_matches", con = connect, if_exists = "replace", index = False)

306

In [30]:
matching_players = pd.read_json("Projet_centres_data/matching_players.json")

In [31]:
matching_players.to_sql("matching_players", con = connect, if_exists = "replace", index = False)

639

In [32]:
matching_teams = pd.read_json("Projet_centres_data/matching_teams.json")

In [33]:
matching_teams.to_sql("matching_teams", con = connect, if_exists = "replace", index = False)

18

In [34]:
SB_matches = pd.read_json("Projet_centres_data/SB_matches.json")

In [35]:
SB_matches.to_sql("SB_matches", con = connect, if_exists = "replace", index = False)

306

In [36]:
import pandas as pd

# id a exclure car pas de correspondance entre SB et SKC
excluded_ids = [3894343, 3894203]

# concatene les matchs en enlevant ceux exclus
events = pd.concat(
    [
        pd.read_json(f"Projet_centres_data/SB_events/{match_id}.json") 
        for match_id in SB_matches.match_id 
        if match_id not in excluded_ids  # exclure les fichiers avec les id supprimes
    ]
)

print(f"Le DataFrame final a {len(events)} lignes.")


C:\Users\leobe\AppData\Local\Temp\ipykernel_36576\1193104715.py:9: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  pd.read_json(f"Projet_centres_data/SB_events/{match_id}.json")
C:\Users\leobe\AppData\Local\Temp\ipykernel_36576\1193104715.py:9: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  pd.read_json(f"Projet_centres_data/SB_events/{match_id}.json")
C:\Users\leobe\AppData\Local\Temp\ipykernel_36576\1193104715.py:9: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsi

Le DataFrame final a 1111228 lignes.


In [37]:
#colonnes a garder
colonnes = ["shot_type", "shot_outcome", "type", "match_id", "period", "possession", "possession_team_id", "play_pattern", "location", "pass_cross", "pass_type", "index", "id",
            "minute", "pass_end_location", "shot_end_location", "pass_body_part"]

events = events[colonnes].reset_index().rename({"index" : "index_event"}, axis = 1)

events_loc = events.pop("location").dropna()
events_loc = pd.DataFrame(events_loc.tolist(), index = events_loc.index, columns = ["x_loc", "y_loc", "z_loc"])

events_pass_loc = events.pop("pass_end_location").dropna()
events_pass_loc = pd.DataFrame(events_pass_loc.tolist(), index = events_pass_loc.index, columns = ["x_pass", "y_pass"])

events_shot_loc = events.pop("shot_end_location").dropna()
events_shot_loc = pd.DataFrame(events_shot_loc.tolist(), index = events_shot_loc.index, columns = ["x_shot", "y_shot", "z_shot"])

events = pd.concat([events, events_loc, events_pass_loc, events_shot_loc], axis = 1)

In [38]:
events.to_sql("events", con = connect, if_exists = "replace", index = False)

1111228

In [39]:
print(list(events.columns))

['level_0', 'shot_type', 'shot_outcome', 'type', 'match_id', 'period', 'possession', 'possession_team_id', 'play_pattern', 'pass_cross', 'pass_type', 'index_event', 'id', 'minute', 'pass_body_part', 'x_loc', 'y_loc', 'z_loc', 'x_pass', 'y_pass', 'x_shot', 'y_shot', 'z_shot']


In [ ]:
import json
import os
import pandas as pd

matching_file_path = "Projet_centres_data/matching_matches.json"  
frame_folder_path = "Projet_centres_data/SKC_crosses_freeze_frames"

# identifiants SKC des fichiers invalides à exclure
invalid_skillcorner_ids = {1510819, 1299412, 1160044}

# pour charger les fichiers json
try:
    with open(matching_file_path, 'r') as f:
        match_mapping = json.load(f)
    print(f"Le fichier {matching_file_path} a été ouvert et chargé avec succès.")
    print(f"Nombre de correspondances trouvées : {len(match_mapping)}")  # Afficher le nombre de correspondances
except Exception as e:
    print(f"Erreur lors de l'ouverture ou du chargement du fichier {matching_file_path}: {e}")
    match_mapping = []  # liste si le fichier n'a pas pu être chargé

# liste pour stocker les chemins de fichiers
files_to_read = []

# parcourir chaque correspondance dans la liste
for match in match_mapping:
    sc_id = match["skillcorner_id"]  # id SKC

    # ignorer les SKC invalides
    if sc_id in invalid_skillcorner_ids:
        print(f"Fichier ignoré pour SkillCorner ID: {sc_id} (StatsBomb ID: {match['statsbomb_id']})")
        continue

    # construire le chemin vers le fichier Skill Corner (Freeze Frame)
    sc_file_path = os.path.join(frame_folder_path, f"{sc_id}.json")

    # ajouter le chemin du fichier à la liste s'il existe
    if os.path.exists(sc_file_path):
        files_to_read.append(sc_file_path)
    else:
        print(f"Fichier manquant pour SkillCorner ID: {sc_id} (StatsBomb ID: {match['statsbomb_id']})")

# concatener les fichiers JSON directement dans un DataFrame
if files_to_read:
    frame = pd.concat([pd.read_json(file) for file in files_to_read], ignore_index=True)
    print(f"Le DataFrame final a {len(frame)} lignes.")
else:
    print("Aucune donnée n'a été ajoutée au DataFrame.")


Le fichier Projet_centres_data/matching_matches.json a été ouvert et chargé avec succès.
Nombre de correspondances trouvées : 306
Fichier ignoré pour SkillCorner ID: 1510819 (StatsBomb ID: 3894343)
Fichier ignoré pour SkillCorner ID: 1299412 (StatsBomb ID: 3894203)
Fichier ignoré pour SkillCorner ID: 1160044 (StatsBomb ID: None)
Le DataFrame final a 20576 lignes.


In [ ]:
import pandas as pd

# identification des centres (pass_cross= 1 et pass_type != Corner, Throw In, Free Kick)
# trouver les indices des centres valides
centres_indices = events[
    (events['pass_cross'] == 1) & (~events['pass_type'].isin(['Corner', 'Throw In', 'Free Kick']))
].index.tolist()

# afficher les indices trouvés pour verif
print(f"Nombre de centres valides trouvés : {len(centres_indices)}")

centres_ids = events.loc[centres_indices, 'id'].tolist() 

# verif : si aucun centre trouvé, afficher erreur
if not centres_indices:
    print("Aucun centre valide trouvé dans le DataFrame.")
else:
    print(f"{len(centres_indices)} centres identifiés.")

result_columns = ['centre_event_id', 'centre_event_type']
for i in range(1, 6):
    result_columns.extend([f'action_{i}_id', f'action_{i}_type'])

#liste de resultats
result_data = []

# parcourir centres et associer les 5 événements suivants avec leurs types
for centre_index in centres_indices:
    # verifier qu'il y a au moins 5 événements après le centre dans events
    if centre_index + 2 + 5 <= len(events):  # +2 pour ignorer l'événement centre
        centre_event_id = events.loc[centre_index, 'id']  # id de l'événement du centre
        centre_event_type = events.loc[centre_index, 'type']  # type de l'événement centre (normalement pass si pas d'erreurs)
        actions_suivantes = events.iloc[centre_index + 2: centre_index + 7]  # récupérer les 5 événements suivants
        
        # filtre pour exclure les centres dans les 5 actions suivantes
        actions_suivantes_non_centres = actions_suivantes[~actions_suivantes['id'].isin(centres_ids)]
        
        # si on a moins de 5 evenements non-centres après ce centre, on le considère comme invalide
        if len(actions_suivantes_non_centres) >= 5:
            # prendre les 5 événements non-centres et leurs types
            actions_suivantes_finales = actions_suivantes_non_centres[['id', 'type', 'shot_type', 'pass_type']].iloc[:5]
            
            row_data = [centre_event_id, centre_event_type]
            for _, row in actions_suivantes_finales.iterrows():
                row_data.append(row['id']) 
                if row['type'] == 'Shot':
                    if pd.isna(row['shot_type']):  # si shot_type est NaN, on conserve 'Shot'
                        row_data.append('Shot')  
                    else:
                        row_data.append(row['shot_type'])  # si pas NaN, utiliser shot_type pour + de precision
                elif row['type'] == 'Pass':
                    if pd.isna(row['pass_type']):  # si pass_type est NaN, conserver 'Pass'
                        row_data.append('Pass')
                    else:
                        row_data.append(row['pass_type'])  # si pas NaN, utiliser pass_type
                else:
                    row_data.append(row['type'])  # sinon, utiliser le type directement
            
            result_data.append(row_data)

centres_actions_with_types_df = pd.DataFrame(result_data, columns=result_columns)

#affichage
if not centres_actions_with_types_df.empty:
    print(f"Il y a {len(centres_actions_with_types_df)} centres valides dans le DataFrame.")
    print(centres_actions_with_types_df.head(10).to_string(index=False))  # afficher les 10 premières lignes sans index pour meilleur affichage


Nombre de centres valides trouvés : 6346
6346 centres identifiés.
Il y a 5655 centres valides dans le DataFrame.
                     centre_event_id centre_event_type                          action_1_id action_1_type                          action_2_id action_2_type                          action_3_id action_3_type                          action_4_id action_4_type                          action_5_id action_5_type
93400424-7906-40d7-9816-b67439e9c685              Pass b591698c-96b8-4fa0-911d-f1768c33d167      Recovery a01b8352-7f4f-4b89-963f-2fb68f4d7bc8      Recovery 46bdd2eb-cc3f-414a-9c9a-25cc404b56b9          Pass 38709b02-cec8-49fe-bfe0-152843b010c7          Pass dbebaca5-d122-407d-a021-52abf1ebc3c8          Pass
5928926a-9413-4928-859c-f7759878d35c              Pass e56537ab-0b0a-4778-b6a9-17daf6f6d7e7          Pass 41aafc85-aed5-4348-ac96-482476af214e        Corner 4bf84742-4d56-4502-b5de-8f8582cef5ea          Pass 78f782c0-2a23-43d4-a0ab-a7405428a826          Pass 0e7ca715

In [ ]:
import sqlite3

if not centres_actions_with_types_df.empty:
    print(f"Il y a {len(centres_actions_with_types_df)} centres valides dans le DataFrame.")
    print(centres_actions_with_types_df.head(10).to_string(index=False))
    
    # Convertir en base de données SQLite
    conn = sqlite3.connect("centres_actions.db")  # crée ou ouvre un fichier de base de données sqlite
    table_name = "centres_actions_with_types"
    
    # exporter le df dans une table sqlite
    centres_actions_with_types_df.to_sql(table_name, conn, if_exists="replace", index=False)
    print(f"Les données ont été exportées vers la table '{table_name}' dans le fichier 'centres_actions.db'.")
    
    conn.close()
else:
    print("Aucun centre valide avec 5 actions suivantes trouvé.")


Il y a 5655 centres valides dans le DataFrame.
                     centre_event_id centre_event_type                          action_1_id action_1_type                          action_2_id action_2_type                          action_3_id action_3_type                          action_4_id action_4_type                          action_5_id action_5_type
93400424-7906-40d7-9816-b67439e9c685              Pass b591698c-96b8-4fa0-911d-f1768c33d167      Recovery a01b8352-7f4f-4b89-963f-2fb68f4d7bc8      Recovery 46bdd2eb-cc3f-414a-9c9a-25cc404b56b9          Pass 38709b02-cec8-49fe-bfe0-152843b010c7          Pass dbebaca5-d122-407d-a021-52abf1ebc3c8          Pass
5928926a-9413-4928-859c-f7759878d35c              Pass e56537ab-0b0a-4778-b6a9-17daf6f6d7e7          Pass 41aafc85-aed5-4348-ac96-482476af214e        Corner 4bf84742-4d56-4502-b5de-8f8582cef5ea          Pass 78f782c0-2a23-43d4-a0ab-a7405428a826          Pass 0e7ca715-d004-4641-a53c-6322b7fcd342          Pass
32bf3011-261f-4855-aae6

In [ ]:
# Supprimer les colonnes inutiles de frame
frame = frame.drop(columns=['data', 'possession', 'image_corners_projection', 'timestamp'])

# Afficher les colonnes restantes pour vérification
print("Colonnes restantes dans frame après suppression :")
print(frame.columns)


Colonnes restantes dans frame après suppression :
Index(['data', 'possession', 'frame', 'image_corners_projection', 'timestamp',
       'period', 'event_id', 'event_x', 'event_y', 'is_matched'],
      dtype='object')


In [ ]:
import pandas as pd

# filtrer les centres valides à partir de centres_actions_with_types_dfs
centres_valides_ids = centres_actions_with_types_df['centre_event_id'].unique()

# avant la jointure, supprimer les doublons dans frame (normalement inutile mais précaution)
frame_unique_centres = frame.drop_duplicates(subset=['event_id'])

# faire à la jointure avec centres_valides_ids
frame_centre_valide = frame_unique_centres[frame_unique_centres['event_id'].isin(centres_valides_ids)]

# verifier que le nombre de lignes dans frame_centre_valide correspond à celui des centres valides
print(f"Le DataFrame 'frame_centre_valide' contient {len(frame_centre_valide)} lignes.")
print(f"Il y a {len(centres_valides_ids)} centres uniques valides.")

#faire jointure entre frame_centre_valide et frame pour récupérer uniquement les frames des centres valides
frame_final = frame.merge(frame_centre_valide[['event_id']], on='event_id', how='inner')

# verifier la taille et afficher les premières lignes du df final
print(f"Le DataFrame final contient {len(frame_final)} lignes.")
print("Affichage des premières lignes du DataFrame final :")
#print(frame_final.head())

# afficher les colonnes du df pour vérif
print("Colonnes du DataFrame final :")
print(frame_final.columns)

#print(frame.columns)
#print(events.columns)


Le DataFrame 'frame_centre_valide' contient 5509 lignes.
Il y a 5655 centres uniques valides.
Le DataFrame final contient 5509 lignes.
Affichage des premières lignes du DataFrame final :
Colonnes du DataFrame final :
Index(['data', 'possession', 'frame', 'image_corners_projection', 'timestamp',
       'period', 'event_id', 'event_x', 'event_y', 'is_matched'],
      dtype='object')


In [ ]:
# créer une liste pour stocker les données transformées
player_positions = []

# parcourir chaque ligne de frame
for idx, row in frame.iterrows():
    frame_id = row['frame']  
    data = row['data']       # extraire la liste des joueurs (dictionnaire)

    for player_data in data:
        player_positions.append({
            'frame_id': frame_id,
            'track_id': player_data['track_id'],
            'x': player_data['x'],
            'y': player_data['y'],
            'vx': player_data.get('vx', None),  # vitesse sur l'axe X
            'vy': player_data.get('vy', None),  # vitesse sur l'axe Y
            'speed_norm': player_data.get('speed_norm', None),  # vitesse (en norme)
            'accel_norm': player_data.get('accel_norm', None),  # accélération (en norme)
            'is_visible': player_data.get('is_visible', None)   # visibilité du joueur
        })

#On verra si toutes les données sont utiles mais pour l'instant je laisse tout

# convertir la liste en DataFrame
player_positions_df = pd.DataFrame(player_positions)

# verif du df
print("\nDataFrame des positions des joueurs :")
print(player_positions_df.head())

# verif des dimensions
print(f"\nNombre total de lignes dans le DataFrame : {len(player_positions_df)}")



DataFrame des positions des joueurs :
   frame_id  track_id      x      y        vx        vy  speed_norm  \
0      8925      6766  49.24   1.73  1.098593 -0.181956    1.113559   
1      8925      5799  34.32  28.03  3.441184 -0.866181    3.548523   
2      8925     13348  32.14  13.61  3.137006 -0.480590    3.173605   
3      8925     24062  31.16   3.23  3.505434  0.792138    3.593821   
4      8925      5313  28.97  -7.47  3.551075  0.946061    3.674938   

   accel_norm  is_visible  
0   -0.929891       False  
1   -1.794100        True  
2   -1.815245        True  
3   -1.583512        True  
4   -0.860150        True  

Nombre total de lignes dans le DataFrame : 467914


In [ ]:
#Pour détérminer de quel côté doit marquer chaque joueur

    
#arguments : 
#period : La période du match (1 ou 2)
#home_team : 1 si l'équipe est à domicile, 0 sinon

#retourne :
#int : La position x du but où l'équipe doit marquer (0 ou 120)

#je n'ai pas reussi a trouver une correspondance entre les noms et les id des équipes (je pense qu'il manque des infos pour joindre les deux)
#probleme :
#SB_lineups n'a que l'id de chaque équipe (avec les players_id) mais n'a pas le nom de l'équipe rattaché à cet id.
#Mais dans SB_matches y a le nom des équipes à domicile et extérieur mais pas leur id. Donc impossible de faire jointure entre les noms et les id.

# si on arrive a joindre les deux, cette fonction sera utile pour determiner pour chaque joueur s'il joue a domicile ou a l'exterieur
#on pourra donc determiner dans quel but il doit marquer donc ca sera utile pour determiner le nombre de joueurs entre le ballon et le but
        

def goal_side(period, home_team):
    if period == 1:  # 1ère mi-temps
        return 120 if home_team == 1 else 0
    elif period == 2:  # 2ème mi-temps
        return 0 if home_team == 1 else 120


In [ ]:
#MARCHE


# nettoyer frame_final pour garder uniquement les colonnes nécessaires
frame_final_cleaned = frame_final[['event_id', 'frame']]
print("Frame Final Cleaned :")
print(frame_final_cleaned.head())

# ajouter le match_id à frame_final en le joignant avec events
frame_final_with_match = frame_final_cleaned.merge(
    events[['id', 'match_id', 'x_loc', 'y_loc']],  # inclure 'x_loc' et 'y_loc' pour chaque centre
    left_on='event_id',          # correspondance avec event_id dans frame_final
    right_on='id',               # correspondance avec id dans events
    how='left'                   # left join pour garder toutes les lignes de frame_final
).rename(columns={'x_loc': 'x_loc_event', 'y_loc': 'y_loc_event'})  # Renommer les colonnes pour éviter des pb d'ambiguité
frame_final_with_match.drop(columns=['id'], inplace=True)  # supprimer la colonne redondante 'id' (on a deux fois l'id de l'evenement sinon apres jointure)

print("\nFrame Final with Match ID and Event Coordinates:")
print(frame_final_with_match.head())

# nettoyer frame pour inclure match_id en joignant avec events
frame_with_match = frame.merge(
    events[['id', 'match_id', 'x_loc', 'y_loc']],  # inclure 'x_loc' et 'y_loc'
    left_on='event_id',
    right_on='id',
    how='left'
).drop(columns=['id'])

print("\nFrame with Match ID and Coordinates:")
print(frame_with_match.head())

# fonction pour trouver l'événement précédent et sa frame
def get_prev_event_data(match_id, frame):
    # filtrer les frames du même match (meme match_id)
    frames_in_match = frame_with_match[frame_with_match['match_id'] == match_id]

    # trouver la frame directement inférieure a celle du centre
    prev_frames = frames_in_match[frames_in_match['frame'] < frame].sort_values(by='frame', ascending=False)
    if not prev_frames.empty:
        prev_event = prev_frames.iloc[0]  # ligne correspondant à l'évenement précedent
        return prev_event['event_id'], prev_event['frame']  # retourner l'event_id et la frame de l'evenement précédent
    return None, None  # si aucun evenement precedent est trouvé

# appliquer la fonction pour trouver l'événement précédent et sa frame
frame_final_with_match[['prev_event_id', 'prev_frame']] = frame_final_with_match.apply(
    lambda row: pd.Series(get_prev_event_data(row['match_id'], row['frame'])), axis=1
)

# remplacer les valeurs manquantes par 'Unknown'
# en gros il y a des centres pour lesquels l'evenement precedent n'est pas trouvé a cause de manquements dans les données SB (29 sur quasiment 6000 centres donc ok)
#  donc on le met en 'Unknown' pour pas que la db ait de valeurs manquantes
frame_final_with_match['prev_event_id'].fillna('Unknown', inplace=True)
frame_final_with_match['prev_frame'].fillna('Unknown', inplace=True)

# verification après ajout de prev_event_id et prev_frame
print("\nFrame Final with Previous Event ID and Frame:")
print(frame_final_with_match.head())

# jointure avec events pour ajouter les coordonnées et le type de l'événement précédent
frame_final_with_match = frame_final_with_match.merge(
    events[['id', 'type', 'x_loc', 'y_loc']],  
    left_on='prev_event_id',
    right_on='id',
    how='left',
    suffixes=('', '_prev')
).rename(columns={'x_loc': 'x_loc_prev', 'y_loc': 'y_loc_prev'})  # renommage des colonnes pour les coordonnées précédentes
frame_final_with_match.drop(columns=['id'], inplace=True)

# remplacer les valeurs manquantes dans type par 'Unknown'
# memes raisons que le 'Unknown' juste au dessus
frame_final_with_match['type'].fillna('Unknown', inplace=True)

#garder les colonnes necessaires
final_df = frame_final_with_match[
    [
        'event_id', 'match_id', 'frame', 'x_loc_event', 'y_loc_event', 
        'prev_event_id', 'prev_frame', 'x_loc_prev', 'y_loc_prev', 'type'
    ]
]
final_df.rename(columns={'type': 'type_prev'}, inplace=True)

# verif des résultats
print("\nFinal DataFrame with Event, Previous Event Coordinates, and Frame:")
print(final_df.head())

# verif des valeurs manquantes
print("\nMissing Values in Final DataFrame:")
print(final_df.isnull().sum())

#convertir les colonnes pour s'assurer qu'elles ont le même type (conversion de prev_frame et frame_id au meme type)
final_df['prev_frame'] = pd.to_numeric(final_df['prev_frame'], errors='coerce')  # Convertir en numérique
player_positions_df['frame_id'] = pd.to_numeric(player_positions_df['frame_id'], errors='coerce') 


# jointure de final_df avec player_positions_df pour inclure les positions des joueurs lors de la frame de l'événement précédent chaque centre
final_df_with_players = final_df.merge(
    player_positions_df,                # df des positions des joueurs
    left_on='prev_frame',               # frame précédent le centre
    right_on='frame_id',                # correspondance avec les frames de player_positions_df
    how='left',                         # left join pour conserver toutes les lignes de final_df
    suffixes=('', '_player')            # pour pas qu'il y ait de pb de conflits de noms entre les colonnes des differents df
)

# verif du résultat après la jointure
print("\nFinal DataFrame avec positions des joueurs :")
print(final_df_with_players.head())

print(f"\nNombre total de lignes dans le DataFrame final : {len(final_df_with_players)}")


Frame Final Cleaned :
                               event_id  frame
0  367a4054-3c77-4189-b353-324582b60327   8959
1  7d0f2b1e-716c-4a6f-ba6e-ecc308e371d1   9402
2  31ce184d-479e-4bae-9b86-436707961cf6  11136
3  9494c477-6417-4f3b-93b3-2aff8899587a  11165
4  8a6c8831-a2bd-400b-b540-9db983862d78  21723

Frame Final with Match ID and Event Coordinates:
                               event_id  frame  match_id  x_loc_event  \
0  367a4054-3c77-4189-b353-324582b60327   8959   3894366         87.4   
1  7d0f2b1e-716c-4a6f-ba6e-ecc308e371d1   9402   3894366        109.8   
2  31ce184d-479e-4bae-9b86-436707961cf6  11136   3894366        106.4   
3  9494c477-6417-4f3b-93b3-2aff8899587a  11165   3894366        119.3   
4  8a6c8831-a2bd-400b-b540-9db983862d78  21723   3894366        110.8   

   y_loc_event  
0         12.6  
1         72.0  
2         75.0  
3         28.9  
4         78.4  

Frame with Match ID and Coordinates:
                                                data  \
0  [{'track

C:\Users\leobe\AppData\Local\Temp\ipykernel_36576\3205751731.py:50: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  frame_final_with_match['prev_event_id'].fillna('Unknown', inplace=True)
C:\Users\leobe\AppData\Local\Temp\ipykernel_36576\3205751731.py:51: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always be


Frame Final with Previous Event ID and Frame:
                               event_id  frame  match_id  x_loc_event  \
0  367a4054-3c77-4189-b353-324582b60327   8959   3894366         87.4   
1  7d0f2b1e-716c-4a6f-ba6e-ecc308e371d1   9402   3894366        109.8   
2  31ce184d-479e-4bae-9b86-436707961cf6  11136   3894366        106.4   
3  9494c477-6417-4f3b-93b3-2aff8899587a  11165   3894366        119.3   
4  8a6c8831-a2bd-400b-b540-9db983862d78  21723   3894366        110.8   

   y_loc_event                         prev_event_id prev_frame  
0         12.6  bef00848-b591-4e9f-b0cc-7d2a8013c7b9     8925.0  
1         72.0  bf2790f8-81d4-4819-b15c-b3abd1f2955d     9396.0  
2         75.0  8102c535-ddfe-4aa4-b245-aaa99aa6106a    11130.0  
3         28.9  431f16eb-50f4-47c3-8e2f-e4a7e2f74bd3    11152.0  
4         78.4  d90c0012-f50e-4ba7-b954-0283d870336b    21715.0  

Final DataFrame with Event, Previous Event Coordinates, and Frame:
                               event_id  match_id 

C:\Users\leobe\AppData\Local\Temp\ipykernel_36576\3205751731.py:68: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  frame_final_with_match['type'].fillna('Unknown', inplace=True)


In [50]:
import sqlite3

#creation de la db avec la position de tous les joueurs a chaque frame aussi
db_name = "centres_positions_4.db"  
table_name = "event_positions"  

conn = sqlite3.connect(db_name)

final_df_with_players.to_sql(table_name, conn, if_exists='replace', index=False)
print(f"\nLes données ont été enregistrées dans la base de données '{db_name}' sous la table '{table_name}'.")


conn.close() 



Les données ont été enregistrées dans la base de données 'centres_positions_4.db' sous la table 'event_positions'.
